# Análisis Cartola Débito Directo
**Período:** 08 Agosto - 14 Septiembre 2022

## Flujo del Producto PAC
1. **Ingreso por PAC (+)**: Bancos debitan a usuarios y transfieren a Fintoc → dinero ENTRA
2. **Liquidación a Comercio (-)**: Fintoc paga a comercios al día hábil siguiente → dinero SALE

In [15]:
import pandas as pd
import json
import os
from datetime import datetime, timedelta

# Cargar datos
df = pd.read_csv('inputs/debito_directo.csv')
df['date'] = pd.to_datetime(df['date'])

# Clasificar movimientos
df['tipo'] = df['description'].apply(lambda x: 'ingreso' if 'Ingreso por PAC' in x else 'liquidacion')
df['banco'] = df['description'].apply(lambda x: x.replace('Ingreso por PAC Multibanco ', '').replace('Ingreso por PAC Multibco.', '') if 'Ingreso' in x else None)
df['comercio'] = df['description'].apply(lambda x: x.replace('Liquidacion a: ', '') if 'Liquidacion' in x else None)

print(f"Período: {df['date'].min().date()} al {df['date'].max().date()}")
print(f"Total registros: {len(df)}")

Período: 2022-08-08 al 2022-09-14
Total registros: 70


In [16]:
# Separar ingresos y liquidaciones
ingresos = df[df['tipo'] == 'ingreso']
liquidaciones = df[df['tipo'] == 'liquidacion']

total_ingresos = ingresos['amount'].sum()
total_liquidaciones = abs(liquidaciones['amount'].sum())
descuadre = total_ingresos - total_liquidaciones

print("=" * 50)
print("RESUMEN FINANCIERO")
print("=" * 50)
print(f"Ingresos PAC:    ${total_ingresos:>12,} CLP")
print(f"Liquidaciones:   ${total_liquidaciones:>12,} CLP")
print(f"Descuadre:       ${descuadre:>12,} CLP")

if descuadre < 0:
    print(f"\n⚠️ ALERTA: Las liquidaciones superan los ingresos en ${abs(descuadre):,}")

RESUMEN FINANCIERO
Ingresos PAC:    $  15,342,256 CLP
Liquidaciones:   $  15,352,260 CLP
Descuadre:       $     -10,004 CLP

⚠️ ALERTA: Las liquidaciones superan los ingresos en $10,004


In [17]:
# Análisis por banco
por_banco = ingresos.groupby('banco').agg(
    total=('amount', 'sum'),
    cantidad=('amount', 'count')
).sort_values('total', ascending=False)

print("\nINGRESOS POR BANCO")
print("-" * 50)
for banco, row in por_banco.iterrows():
    pct = row['total'] / total_ingresos * 100
    print(f"{banco:40} ${row['total']:>10,} ({pct:.1f}%)")


INGRESOS POR BANCO
--------------------------------------------------
Banco Santander                          $ 5,961,579 (38.9%)
bco.credito Inversiones                  $ 4,251,225 (27.7%)
Banco Del Estado                         $ 2,270,154 (14.8%)
Bank Boston N.a.                         $ 1,392,341 (9.1%)
Banco Falabella                          $   848,250 (5.5%)
Banco Sud Americano                      $   618,707 (4.0%)


In [18]:
# Análisis por comercio
por_comercio = liquidaciones.groupby('comercio').agg(
    total=('amount', lambda x: abs(x.sum())),
    cantidad=('amount', 'count')
).sort_values('total', ascending=False)

print("\nLIQUIDACIONES POR COMERCIO")
print("-" * 50)
for comercio, row in por_comercio.iterrows():
    pct = row['total'] / total_liquidaciones * 100
    print(f"{comercio:40} ${row['total']:>10,} ({pct:.1f}%)")


LIQUIDACIONES POR COMERCIO
--------------------------------------------------
NoPayments                               $10,657,027 (69.4%)
Tesla                                    $ 1,353,581 (8.8%)
Mirador San Juan                         $ 1,342,810 (8.7%)
Seguros Asesorados Ltda                  $   979,536 (6.4%)
Constructora e Inmobiliaria Partners     $   575,640 (3.7%)
Inmobiliaria Ruiz-Tagle SPA              $   237,346 (1.5%)
SALON SPA ESPINOSA                       $   186,165 (1.2%)
Roots Housing SPA                        $     8,265 (0.1%)
Smart Finances SpA                       $     8,265 (0.1%)
Roots Housing Spa                        $     1,876 (0.0%)
Smart Finances SPA                       $     1,739 (0.0%)
Costanera Chacabuco Ltda                 $         6 (0.0%)
PZ Construcciones S.A.                   $         4 (0.0%)


In [19]:
# Detectar anomalías
anomalias = []

# 1. Descuadre general
if descuadre != 0:
    anomalias.append({
        'tipo': 'Descuadre financiero',
        'descripcion': f'Diferencia de ${abs(descuadre):,} entre ingresos y liquidaciones',
        'impacto': abs(descuadre),
        'severidad': 'alta'
    })

# 2. Montos muy pequeños (<$50)
montos_pequenos = df[df['amount'].abs() < 50]
if len(montos_pequenos) > 0:
    anomalias.append({
        'tipo': 'Movimientos atípicos',
        'descripcion': f'{len(montos_pequenos)} transacciones con montos < $50 (posibles pruebas)',
        'impacto': len(montos_pequenos),
        'severidad': 'baja',
        'detalle': montos_pequenos[['date', 'description', 'amount']].to_dict('records')
    })

# 3. Comercios con nombres duplicados
comercios_normalizados = {}
for c in por_comercio.index:
    key = c.lower().strip()
    if key not in comercios_normalizados:
        comercios_normalizados[key] = []
    comercios_normalizados[key].append(c)

duplicados = {k: v for k, v in comercios_normalizados.items() if len(v) > 1}
if duplicados:
    anomalias.append({
        'tipo': 'Datos inconsistentes',
        'descripcion': 'Comercios con variantes de nombre',
        'impacto': 0,
        'severidad': 'media',
        'detalle': duplicados
    })

# 4. Liquidaciones del 31/08 sin ingreso correspondiente
ing_30 = ingresos[ingresos['date'] == '2022-08-30']['amount'].sum()
liq_31 = abs(liquidaciones[liquidaciones['date'] == '2022-08-31']['amount'].sum())
if liq_31 > ing_30:
    anomalias.append({
        'tipo': 'Flujo incorrecto',
        'descripcion': f'31/08: Se liquidaron ${liq_31:,} pero ingresaron solo ${ing_30:,} el 30/08',
        'impacto': liq_31 - ing_30,
        'severidad': 'alta'
    })

print("\nANOMALÍAS DETECTADAS")
print("=" * 50)
for i, a in enumerate(anomalias, 1):
    print(f"\n{i}. [{a['severidad'].upper()}] {a['tipo']}")
    print(f"   {a['descripcion']}")


ANOMALÍAS DETECTADAS

1. [ALTA] Descuadre financiero
   Diferencia de $10,004 entre ingresos y liquidaciones

2. [BAJA] Movimientos atípicos
   12 transacciones con montos < $50 (posibles pruebas)

3. [MEDIA] Datos inconsistentes
   Comercios con variantes de nombre

4. [ALTA] Flujo incorrecto
   31/08: Se liquidaron $10,030 pero ingresaron solo $40 el 30/08


In [20]:
# Generar outputs consolidados
os.makedirs('outputs', exist_ok=True)

# Función para convertir tipos numpy a Python nativo
def to_native(val):
    if hasattr(val, 'item'):
        return val.item()
    return val

# JSON consolidado
reporte = {
    'metadata': {
        'periodo': {'inicio': '2022-08-08', 'fin': '2022-09-14'},
        'generado': datetime.now().isoformat(),
        'total_registros': len(df)
    },
    'resumen_financiero': {
        'total_ingresos': to_native(total_ingresos),
        'total_liquidaciones': to_native(total_liquidaciones),
        'descuadre': to_native(descuadre),
        'num_ingresos': len(ingresos),
        'num_liquidaciones': len(liquidaciones)
    },
    'por_banco': [
        {'banco': banco, 'total': to_native(row['total']), 'cantidad': to_native(row['cantidad'])}
        for banco, row in por_banco.iterrows()
    ],
    'por_comercio': [
        {'comercio': comercio, 'total': to_native(row['total']), 'cantidad': to_native(row['cantidad'])}
        for comercio, row in por_comercio.iterrows()
    ],
    'anomalias': [
        {'tipo': a['tipo'], 'descripcion': a['descripcion'], 'impacto': to_native(a['impacto']), 'severidad': a['severidad']}
        for a in anomalias
    ]
}

with open('outputs/reporte_consolidado.json', 'w', encoding='utf-8') as f:
    json.dump(reporte, f, indent=2, ensure_ascii=False)

print("✓ Generado: outputs/reporte_consolidado.json")

✓ Generado: outputs/reporte_consolidado.json


In [21]:
# TXT con conclusiones
conclusiones = f"""CONCLUSIONES DEL ANÁLISIS - DÉBITO DIRECTO
Período: 08 Agosto - 14 Septiembre 2022
{'=' * 60}

RESUMEN FINANCIERO
-----------------
• Ingresos PAC:    ${total_ingresos:>12,} CLP ({len(ingresos)} movimientos)
• Liquidaciones:   ${total_liquidaciones:>12,} CLP ({len(liquidaciones)} movimientos)
• Descuadre:       ${descuadre:>12,} CLP

RESPUESTA A LAS PREGUNTAS
-------------------------

1. ¿SE CONDICE EL FUNCIONAMIENTO DEL PRODUCTO CON LOS MOVIMIENTOS?

   PARCIALMENTE. El flujo esperado es:
   - Día N: Banco transfiere a Fintoc (Ingreso PAC)
   - Día N+1 hábil: Fintoc paga a comercios (Liquidación)
   
   Este patrón se cumple en la mayoría de casos, pero hay excepciones
   que generan un descuadre de ${abs(descuadre):,} CLP.

2. ANOMALÍAS ENCONTRADAS

   a) DESCUADRE DE ${abs(descuadre):,} CLP (SEVERIDAD ALTA)
      Las liquidaciones superan a los ingresos. El 31/08 se liquidaron
      ${int(liq_31):,} CLP pero solo entraron ${int(ing_30):,} CLP el día anterior.
   
   b) {len(montos_pequenos)} MOVIMIENTOS ATÍPICOS (SEVERIDAD BAJA)
      Transacciones con montos menores a $50, posiblemente pruebas
      de sistema o errores de digitación.
   
   c) COMERCIOS CON NOMBRES DUPLICADOS (SEVERIDAD MEDIA)
      Existen variantes de nombres (ej: "Spa" vs "SPA") que
      dificultan la conciliación correcta.

RECOMENDACIONES
---------------
1. Investigar las liquidaciones del 31/08 que no tienen ingreso asociado
2. Normalizar nombres de comercios en el sistema
3. Revisar si los movimientos pequeños son pruebas de sistema
"""

with open('outputs/conclusiones.txt', 'w', encoding='utf-8') as f:
    f.write(conclusiones)

print("✓ Generado: outputs/conclusiones.txt")
print("\n" + conclusiones)

✓ Generado: outputs/conclusiones.txt

CONCLUSIONES DEL ANÁLISIS - DÉBITO DIRECTO
Período: 08 Agosto - 14 Septiembre 2022

RESUMEN FINANCIERO
-----------------
• Ingresos PAC:    $  15,342,256 CLP (26 movimientos)
• Liquidaciones:   $  15,352,260 CLP (44 movimientos)
• Descuadre:       $     -10,004 CLP

RESPUESTA A LAS PREGUNTAS
-------------------------

1. ¿SE CONDICE EL FUNCIONAMIENTO DEL PRODUCTO CON LOS MOVIMIENTOS?

   PARCIALMENTE. El flujo esperado es:
   - Día N: Banco transfiere a Fintoc (Ingreso PAC)
   - Día N+1 hábil: Fintoc paga a comercios (Liquidación)

   Este patrón se cumple en la mayoría de casos, pero hay excepciones
   que generan un descuadre de $10,004 CLP.

2. ANOMALÍAS ENCONTRADAS

   a) DESCUADRE DE $10,004 CLP (SEVERIDAD ALTA)
      Las liquidaciones superan a los ingresos. El 31/08 se liquidaron
      $10,030 CLP pero solo entraron $40 CLP el día anterior.

   b) 12 MOVIMIENTOS ATÍPICOS (SEVERIDAD BAJA)
      Transacciones con montos menores a $50, posiblem

In [22]:
# Generar email para el encargado del producto
email = f"""Asunto: Cierre Contable Débito Directo - Agosto/Septiembre 2022

Hola,

Adjunto el análisis de la cartola de Débito Directo para el período del 08 de agosto al 14 de septiembre.

RESUMEN EJECUTIVO
-----------------
• Ingresos PAC:    ${total_ingresos:,} CLP
• Liquidaciones:   ${total_liquidaciones:,} CLP  
• Descuadre:       ${abs(descuadre):,} CLP (liquidaciones > ingresos)

RESPUESTA A TUS PREGUNTAS

1. ¿Se condice el funcionamiento del producto con los movimientos?

En general SÍ, pero con excepciones. El flujo esperado (ingreso día N → liquidación día N+1 hábil) 
se cumple en la mayoría de los casos. Sin embargo, encontré situaciones donde se liquidó dinero 
que no había ingresado previamente, lo cual genera el descuadre.

2. ¿Encuentras alguna anomalía?

Sí, encontré las siguientes:

a) DESCUADRE DE ${abs(descuadre):,} CLP [URGENTE]
   El 31 de agosto se liquidaron ${int(liq_31):,} CLP a comercios, pero el día 
   anterior (30/08) solo ingresaron ${int(ing_30):,} CLP. Esto significa que se pagó
   dinero que no había entrado a la cuenta.
   
   Detalle de liquidaciones del 31/08:
   - Seguros Asesorados: $2
   - Mirador San Juan: $4
   - SALON SPA ESPINOSA: $2
   - Inmobiliaria Ruiz-Tagle: $2
   - Smart Finances: $8,265
   - Costanera Chacabuco: $6
   - PZ Construcciones: $4
   - Roots Housing: $1,739
   - Tesla: $6

b) {len(montos_pequenos)} MOVIMIENTOS ATÍPICOS
   Hay transacciones con montos muy pequeños (<$50), incluyendo algunas de $2, $4, $6.
   Podrían ser pruebas de sistema o errores. Sugiero revisar si son intencionales.

c) COMERCIOS CON NOMBRES INCONSISTENTES
   Algunos comercios aparecen con variantes de nombre (ej: "Smart Finances SPA" vs 
   "Smart Finances SpA"). Esto puede dificultar la conciliación y reportería.

PRÓXIMOS PASOS SUGERIDOS
------------------------
1. Investigar el origen del descuadre del 31/08 - ¿de dónde salió ese dinero?
2. Confirmar si los movimientos pequeños son pruebas o errores
3. Normalizar nombres de comercios en el sistema

Quedo atento a tus comentarios.

Saludos,
[Tu nombre]
"""

with open('outputs/email_encargado.txt', 'w', encoding='utf-8') as f:
    f.write(email)

print("✓ Generado: outputs/email_encargado.txt")
print("\n" + "=" * 60)
print(email)

✓ Generado: outputs/email_encargado.txt

Asunto: Cierre Contable Débito Directo - Agosto/Septiembre 2022

Hola,

Adjunto el análisis de la cartola de Débito Directo para el período del 08 de agosto al 14 de septiembre.

RESUMEN EJECUTIVO
-----------------
• Ingresos PAC:    $15,342,256 CLP
• Liquidaciones:   $15,352,260 CLP  
• Descuadre:       $10,004 CLP (liquidaciones > ingresos)

RESPUESTA A TUS PREGUNTAS

1. ¿Se condice el funcionamiento del producto con los movimientos?

En general SÍ, pero con excepciones. El flujo esperado (ingreso día N → liquidación día N+1 hábil) 
se cumple en la mayoría de los casos. Sin embargo, encontré situaciones donde se liquidó dinero 
que no había ingresado previamente, lo cual genera el descuadre.

2. ¿Encuentras alguna anomalía?

Sí, encontré las siguientes:

a) DESCUADRE DE $10,004 CLP [URGENTE]
   El 31 de agosto se liquidaron $10,030 CLP a comercios, pero el día 
   anterior (30/08) solo ingresaron $40 CLP. Esto significa que se pagó
   dinero q

In [23]:
# Resumen de archivos generados
print("\n" + "=" * 60)
print("ARCHIVOS GENERADOS")
print("=" * 60)
print("\n1. outputs/reporte_consolidado.json")
print("   → Datos completos del análisis en formato estructurado")
print("\n2. outputs/conclusiones.txt")
print("   → Resumen ejecutivo con conclusiones principales")
print("\n3. outputs/email_encargado.txt")
print("   → Email listo para enviar al encargado del producto")


ARCHIVOS GENERADOS

1. outputs/reporte_consolidado.json
   → Datos completos del análisis en formato estructurado

2. outputs/conclusiones.txt
   → Resumen ejecutivo con conclusiones principales

3. outputs/email_encargado.txt
   → Email listo para enviar al encargado del producto
